In [1]:
!pip install pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 7.9 MB/s eta 0:00:00


# Imports

In [2]:
import os
import json
import numpy as np
import cv2
import torch
from segment_anything import sam_model_registry, SamPredictor
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import pycocotools.mask as maskUtils
import matplotlib.pyplot as plt
from skimage.draw import polygon
from functools import singledispatch

# Config

In [3]:
CHECKPOINT_PATH = '/kaggle/input/sam/other/default/1/sam_vit_l_0b3195.pth'
VAL_IMAGES_PATH = '/kaggle/input/underwaterimageinstancesegmentation/UIIS/UDW/val'
VAL_ANNOTATIONS_PATH = '/kaggle/input/underwaterimageinstancesegmentation/UIIS/UDW/annotations/val.json'

@singledispatch
def to_serializable(val):
    """Used by default."""
    return str(val)

@to_serializable.register(np.float32)
def ts_float32(val):
    """Used if *val* is an instance of numpy.float32."""
    return float(val)

@to_serializable.register(np.float64)
def ts_float64(val):
    """Used if *val* is an instance of numpy.float64."""
    return float(val)

# Load the Model

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sam = sam_model_registry['vit_l'](checkpoint=CHECKPOINT_PATH).to(device)
predictor = SamPredictor(sam)

/opt/conda/lib/python3.10/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


# Visualization methods

In [5]:
def visualize_sample(image_rgb, gt_mask_combined, mask):
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(image_rgb)
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(gt_mask_combined, cmap='gray')
    plt.title('Ground Truth Mask')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(mask, cmap='gray')
    plt.title('Predicted Mask')
    plt.axis('off')

    plt.show()

# Evaluating

In [6]:
with open(VAL_ANNOTATIONS_PATH) as f:
    coco_data = json.load(f)
coco_gt = COCO(VAL_ANNOTATIONS_PATH)
results = []

for img_idx, img_info in enumerate(coco_data['images']):
    img_id = img_info['id']
    img_path = os.path.join(VAL_IMAGES_PATH, img_info['file_name'])

    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    predictor.set_image(image_rgb)

    boxes = np.array([[0, 0, img_info['width'], img_info['height']]])
    masks, scores, _ = predictor.predict(box=boxes, multimask_output=True)

    gt_mask_combined = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

    ann_ids = coco_gt.getAnnIds(imgIds=img_id)
    gt_annotations = coco_gt.loadAnns(ann_ids)
    
    for annotation in gt_annotations:
        poly_coords = annotation['segmentation'][0]
        rr, cc = polygon(poly_coords[1::2], poly_coords[0::2], gt_mask_combined.shape)
        gt_mask_combined[rr, cc] = 1

    for i in range(len(masks)):
        mask = masks[i]
        score = scores[i]
        if score > 0.5:
            rle = maskUtils.encode(np.asfortranarray(mask))
            rle['counts'] = rle['counts'].decode('utf-8')
            
            results.append({
                'image_id': img_id,
                'category_id': 1,
                'segmentation': rle,
                'score': score,
                'area': np.sum(mask),
            })
            

with open('results.json', 'w') as f:
    json.dump(results, f, default=to_serializable)

coco_dt = coco_gt.loadRes('results.json')
coco_eval = COCOeval(coco_gt, coco_dt, 'segm')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

loading annotations into memory...
Done (t=0.23s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.24s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.80s).
Accumulating evaluation results...
DONE (t=0.11s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets